In [17]:
from tkinter import *
import tkinter as tk
from tkinter.font import Font
from tkinter import messagebox, ttk
from tkinter import PhotoImage
from tkinter import Button
import sqlite3
import re 


In [18]:
class MessageCounselor: # WORKS - DONE!!!
    def __init__(self, db_path, logged_in_user_id):
        """
        Initializes the ability for a patient to message a counselors with a SQLite database connection.

        Parameters:
        - db_path (str): The file path to the SQLite database.
        - logged_in_user_id (int): The ID of the currently logged-in user.
        """
        
        # stores the DB path for reference
        self.db_path = db_path
        # stores the logged-in patient's ID for messages
        self.logged_in_user_id = logged_in_user_id
        
        # connects to DB
        self.conn = sqlite3.connect(db_path)
        # creates cursor object for SQL queries
        self.cursor = self.conn.cursor()
        
        # creates table if it doesn't exist
        self.counselor_chat_table()
        
        # creates chat window and message feed
        self.create_interface()

    def create_interface(self):
        """
        Creates the user interface for the peer chat.
        """
        self.peer = Toplevel()
        self.peer.title("Resource Treatment Center || Counselor Chat")
        self.peer.geometry('800x950')
        self.center_window(800, 950)
        self.peer.resizable(False, False)
        self.peer.config(background="light grey")

        # Create a Canvas widget to hold background image and label/button widgets
        self.canvas = Canvas(self.peer, width=800, height=950)
        self.canvas.pack(fill="both", expand=True)
        # loads image for BK
        self.account_banner = PhotoImage(file="fist-bump.png")
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")

        # default fonts, label specs, and colors for GUI 
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)
        self.label_font_color = "navy"
        self.label_bg = "white"
        self.button_font = Font(family="Helvetica", size=12, weight="bold")        
        self.button_bg = "navy"
        self.button_fg = "light gray"
        self.post_background = "light grey"
        

        # Title table and placement
        self.welcome_label = Label(self.peer, text="RTC || Counselor Chat", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(400, 50, window=self.welcome_label)
        # Sublabel message and placement
        self.detailMessage = Label(self.peer, text="Private message a Counselor anytime!", fg=self.label_font_color, font=self.entry_font, bg=self.label_bg)
        self.canvas.create_window(400, 100, window=self.detailMessage)
        # label for dropdown menu and placement
        self.peer_list_label = Label(self.peer, text="Select a Counselor:", font=self.button_font, bg=self.label_bg, fg=self.label_font_color)
        self.canvas.create_window(100, 150, window=self.peer_list_label)


        # Creating string var to store the selected user's name from dropdown list
        self.selected_counselor = StringVar()
        # set the default value of the dropdown 
        self.selected_counselor.set("Select Counselor")
        # Get list of patients from DB for dropdown list
        users = self.get_counselor_users()
        # create dropdown list with patient names
        self.counselor_dropdown = OptionMenu(self.peer, self.selected_counselor, *users)
        # placement of the dropdown list
        self.canvas.create_window(300, 150, window=self.counselor_dropdown)
        
        # Add a listner to detect when the selected user changes from the default name
        # When user to chat is selected, the load previous message method is called
        self.selected_counselor.trace_add("write", lambda *args: self.load_previous_messages())

      
        # list box creation to display messages
        self.message_feed = Listbox(self.peer, font=self.entry_font, width=50, height=18, bg=self.post_background)
        self.canvas.create_window(400, 375, window=self.message_feed)
        
        # Chat box label and placement
        self.message_entry_label = Label(self.peer, text="Write Message:", font=self.button_font, bg=self.label_bg, fg=self.button_bg)
        self.canvas.create_window(100, 600, window=self.message_entry_label)
        self.message_entry = Entry(self.peer, font=self.entry_font, width=50, bg=self.post_background)
        self.canvas.create_window(400, 600, window=self.message_entry)
        
        # Send message button creating and placement
        self.post_button = Button(self.peer, text="SEND", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.send_counselor_message)
        self.canvas.create_window(700, 600, window=self.post_button)
        
        # Exit button to close the chat window
        self.exit_button = Button(self.peer, text="EXIT", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.peer.destroy)
        self.canvas.create_window(750, 25, window=self.exit_button)

    def counselor_chat_table(self):
        """
        Creates the peer_chat table if it does not exist.
        """
        self.cursor.execute(''' 
            CREATE TABLE IF NOT EXISTS counselor_chat( 
                chat_id INTEGER PRIMARY KEY AUTOINCREMENT,
                sender_id INTEGER NOT NULL,
                receiver_id INTEGER NOT NULL,
                message TEXT NOT NULL,
                is_read INTEGER DEFAULT 0,
                FOREIGN KEY (sender_id) REFERENCES patient_users(p_id),
                FOREIGN KEY (receiver_id) REFERENCES counselor_users(c_id)
            )
        ''')
        self.conn.commit()

    def send_counselor_message(self):
        """
        Sends the message to the selected counselor and updates the message feed.
        """
        
        # get id of logged in user (sender)
        sender_id = self.logged_in_user_id
        # Get selected user info from dropdown
        receiver_info = self.selected_counselor.get()
        # Get user ID of selected user from their info stored in DB
        receiver_id = self.extract_counselor_id(receiver_info)
        # Get message entered by logged in user and remove all leading/trailing spaces
        message = self.message_entry.get().strip()

        # Validate that a counselor has been selected and message is not empty
        if not receiver_id or not message:
            messagebox.showerror("Error", "Please select a counselor and enter a message.")
            return

        # Fetch the sender's username
        self.cursor.execute("SELECT p_username FROM patient_users WHERE p_id = ?", (sender_id,))
        sender_username = self.cursor.fetchone()
        
        if sender_username:
            sender_username = sender_username[0]  # get the senders username
        else:
            sender_username = "Unknown"  # Default if not found

        # Fetch the receiver's username
        self.cursor.execute("SELECT c_username FROM counselor_users WHERE c_id = ?", (receiver_id,))
        # store the receiver's username as receiver_username
        receiver_username = self.cursor.fetchone()
        
        if receiver_username:
            receiver_username = receiver_username[0]  # get the counselor's (receiver) username
        else:
            receiver_username = "Unknown" # default username if not found

        # Insert the message into the database
        self.cursor.execute(
            "INSERT INTO counselor_chat (sender_id, receiver_id, message, is_read) VALUES (?, ?, ?, 0)",
            (sender_id, receiver_id, message)
        )
        self.conn.commit()

        # Add the message to the message feed
        self.message_feed.insert("end", f"{sender_username}: {message}")
        # Clear the chat box to allow the user to enter a new message
        self.message_entry.delete(0, 'end')

    def load_previous_messages(self):
        """
        Loads previous messages between the logged-in user and the selected counselor.
        Retrieves messages from the database and displays them in the message feed 
        using usernames.
        """
        
        # Get the selected peer's information (usually in a format like "Username (ID)")
        receiver_info = self.selected_counselor.get()

        # Extract the user ID of the selected peer from the information
        receiver_id = self.extract_counselor_id(receiver_info)

        # If no receiver is selected or the user is not logged in, do nothing
        if not receiver_id or self.logged_in_user_id is None:
            return

        # Fetch the logged-in user's username from the database
        self.cursor.execute("SELECT p_username FROM patient_users WHERE p_id = ?", (self.logged_in_user_id,))
        sender_username = self.cursor.fetchone()
        sender_username = sender_username[0] if sender_username else "Unknown"  # Extract username or set default

        # Fetch the selected counselor's username from the database
        self.cursor.execute("SELECT c_username FROM counselor_users WHERE c_id = ?", (receiver_id,))
        receiver_username = self.cursor.fetchone()
        receiver_username = receiver_username[0] if receiver_username else "Unknown"  # Extract username or set default

        # Retrieve previous messages exchanged between the logged-in user and the selected peer
        self.cursor.execute(
            """
            SELECT sender_id, message FROM counselor_chat 
            WHERE (sender_id = ? AND receiver_id = ?) 
            OR (sender_id = ? AND receiver_id = ?)
            ORDER BY chat_id ASC
            """, 
            (self.logged_in_user_id, receiver_id, receiver_id, self.logged_in_user_id)
        )

        # Fetch all matching messages from the query
        messages = self.cursor.fetchall()

        # Clear the current message feed to refresh it with new messages
        self.message_feed.delete(0, 'end')

        # Loop through each message and display it in the chat feed
        for sender, message in messages:
            # Determine whether the sender is the logged-in user or the selected friend
            display_name = sender_username if sender == self.logged_in_user_id else receiver_username

            # Insert the message into the message feed with the sender's actual username
            self.message_feed.insert("end", f"{display_name}: {message}")

    def extract_counselor_id(self, user_info):
        """
        Extracts the counselor ID from the selected counselor info (format: 'Firstname Lastname (ID)').
        
        Parameters:
        - user_info (str): The selected counselor's string containing their name and ID.

        Returns:
        - int: The extracted counselor ID.
        - None: If extraction fails due to an incorrect format.
        
        """
        
        try:
            # get ID and remove any trailing spaces
            user_id = int(user_info.split("(")[-1][:-1])
            print(f"Extracted User ID: {user_id} from '{user_info}'")  # Debugging print in terminal
            return user_id
        except (IndexError, ValueError) as e:
            # handles any cases where the input is incorrect
            print(f"Error extracting user ID from '{user_info}': {e}")  # Debugging print in terminal
            return None

    def get_counselor_users(self):
        """
        Retrieves all users from the 'counselor_users' table and formats them as 'Firstname Lastname (ID)'.
        Returns:
            A list of str: A list of counselor users in drop down
        """
        
        # SQL to fetch counselor name and IDs
        self.cursor.execute("SELECT c_id, c_first_name, c_last_name FROM counselor_users")
        # format the data into a row for dropdown list
        return [f"{row[1]} {row[2]} ({row[0]})" for row in self.cursor.fetchall()]

    def center_window(self, width, height):
        """
        Centers the window on the screen.
        """
        screen_width = self.peer.winfo_screenwidth()
        screen_height = self.peer.winfo_screenheight()
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.peer.geometry(f"{width}x{height}+{x}+{y}")

    def close(self):
        """
        Closes the database connection.
        """
        
        self.conn.close()


In [19]:
class PeerChat: # WORKS -- DONE!!!
    def __init__(self, db_path, logged_in_user_id):
        """
        Initializes the PeerChat class with a SQLite database connection.
        
        Websites that helped understand how to complete the chat interface:
        https://stackoverflow.com/questions/42062391/how-to-create-a-chat-window-with-tkinter
        https://www.w3resource.com/python-exercises/tkinter/python-tkinter-layout-management-exercise-7.php
        https://www.sqlite.org/index.html
        
        Parameters:
        - db_path (str): The file path to the SQLite database.
        - logged_in_user_id (int): The ID of the currently logged-in user.
        """
        
        # stores the DB path for reference
        self.db_path = db_path
        # stores the logged-in patient's ID for messages
        self.logged_in_user_id = logged_in_user_id
        
        # connects to DB
        self.conn = sqlite3.connect(db_path)
        # creates cursor object for SQL queries
        self.cursor = self.conn.cursor()
        
        # creates table if it doesn't exist
        self.peer_chat_table()
        
        # creates chat window and message feed
        self.create_interface()

    def create_interface(self):
        """
        Creates the user interface for the peer chat.
        """
        self.peer = Toplevel()
        self.peer.title("Resource Treatment Center || Peer Chat")
        self.peer.geometry('800x950')
        self.center_window(800, 950)
        self.peer.resizable(False, False)
        self.peer.config(background="light grey")

        # Create a Canvas widget to hold background image and label/button widgets
        self.canvas = Canvas(self.peer, width=800, height=950)
        self.canvas.pack(fill="both", expand=True)
        # loads image for BK
        self.account_banner = PhotoImage(file="fist-bump.png")
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")

        # default fonts, label specs, and colors for GUI 
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)
        self.label_font_color = "navy"
        self.label_bg = "white"
        self.button_font = Font(family="Helvetica", size=12, weight="bold")        
        self.button_bg = "navy"
        self.button_fg = "light gray"
        self.post_background = "light grey"
        

        # Title table and placement
        self.welcome_label = Label(self.peer, text="RTC || Peer Chat", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(400, 50, window=self.welcome_label)
        # Sublabel message and placement
        self.detailMessage = Label(self.peer, text="Private message a friend anytime!", fg=self.label_font_color, font=self.entry_font, bg=self.label_bg)
        self.canvas.create_window(400, 100, window=self.detailMessage)
        # label for dropdown menu and placement
        self.peer_list_label = Label(self.peer, text="Select a Friend:", font=self.button_font, bg=self.label_bg, fg=self.label_font_color)
        self.canvas.create_window(100, 150, window=self.peer_list_label)


        # Creating string var to store the selected user's name from dropdown list
        self.selected_user = StringVar()
        # set the default value of the dropdown 
        self.selected_user.set("Select User")
        # Get list of patients from DB for dropdown list
        users = self.get_patient_users()
        # create dropdown list with patient names
        self.peer_dropdown = OptionMenu(self.peer, self.selected_user, *users)
        # placement of the dropdown list
        self.canvas.create_window(300, 150, window=self.peer_dropdown)
        
        # Add a listner to detect when the selected user changes from the default name
        # When user to chat is selected, the load previous message method is called
        self.selected_user.trace_add("write", lambda *args: self.load_previous_messages())


        # list box creation to display messages
        self.message_feed = Listbox(self.peer, font=self.entry_font, width=50, height=18, bg=self.post_background)
        self.canvas.create_window(400, 375, window=self.message_feed)
        
        # Chat box label and placement
        self.message_entry_label = Label(self.peer, text="Write Message:", font=self.button_font, bg=self.label_bg, fg=self.button_bg)
        self.canvas.create_window(100, 600, window=self.message_entry_label)
        self.message_entry = Entry(self.peer, font=self.entry_font, width=50, bg=self.post_background)
        self.canvas.create_window(400, 600, window=self.message_entry)
        
        # Send message button creating and placement
        self.post_button = Button(self.peer, text="SEND", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.send_peer_message)
        self.canvas.create_window(700, 600, window=self.post_button)
        
        # Exit button to close the chat window
        self.exit_button = Button(self.peer, text="EXIT", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.peer.destroy)
        self.canvas.create_window(750, 25, window=self.exit_button)

    def peer_chat_table(self):
        """
        Creates the peer_chat table if it does not exist.
        """
        self.cursor.execute(''' 
            CREATE TABLE IF NOT EXISTS peer_chat( 
                chat_id INTEGER PRIMARY KEY AUTOINCREMENT,
                sender_id INTEGER NOT NULL,
                receiver_id INTEGER NOT NULL,
                message TEXT NOT NULL,
                is_read INTEGER DEFAULT 0,
                FOREIGN KEY (sender_id) REFERENCES patient_users(p_id),
                FOREIGN KEY (receiver_id) REFERENCES patient_users(p_id)
            )
        ''')
        self.conn.commit()

    def send_peer_message(self):
        """
        Sends the message to the selected peer and updates the message feed.
        """
        
        # get id of logged in user (sender)
        sender_id = self.logged_in_user_id
        # Get selected user info from dropdown
        receiver_info = self.selected_user.get()
        # Get user ID of selected user from their info stored in DB
        receiver_id = self.extract_user_id(receiver_info)
        # Get message entered by logged in user and remove all leading/trailing spaces
        message = self.message_entry.get().strip()

        # Validate that a user has been selected and message is not empty
        if not receiver_id or not message:
            messagebox.showerror("Error", "Please select a user and enter a message.")
            return

        # Fetch the sender's username
        self.cursor.execute("SELECT p_username FROM patient_users WHERE p_id = ?", (sender_id,))
        sender_username = self.cursor.fetchone()
        
        if sender_username:
            sender_username = sender_username[0]  # get the senders username
        else:
            sender_username = "Unknown"  # Default if not found

        # Fetch the receiver's username
        self.cursor.execute("SELECT p_username FROM patient_users WHERE p_id = ?", (receiver_id,))
        # store the receiver's username as receiver_username
        receiver_username = self.cursor.fetchone()
        
        if receiver_username:
            receiver_username = receiver_username[0]  # get the peer's (receiver) username
        else:
            receiver_username = "Unknown" # default username if not found

        # Insert the message into the database
        self.cursor.execute(
            "INSERT INTO peer_chat (sender_id, receiver_id, message, is_read) VALUES (?, ?, ?, 0)",
            (sender_id, receiver_id, message)
        )
        self.conn.commit()

        # Add the message to the message feed
        self.message_feed.insert("end", f"{sender_username}: {message}")
        # Clear the chat box to allow the user to enter a new message
        self.message_entry.delete(0, 'end')

    def load_previous_messages(self):
        """
        Loads previous messages between the logged-in user and the selected peer.
        Retrieves messages from the database and displays them in the message feed 
        using actual usernames instead of generic labels like "You" and "Friend".
        """
        
        # Get the selected peer's information (usually in a format like "Username (ID)")
        receiver_info = self.selected_user.get()

        # Extract the user ID of the selected peer from the information
        receiver_id = self.extract_user_id(receiver_info)

        # If no receiver is selected or the user is not logged in, do nothing
        if not receiver_id or self.logged_in_user_id is None:
            return

        # Fetch the logged-in user's username from the database
        self.cursor.execute("SELECT p_username FROM patient_users WHERE p_id = ?", (self.logged_in_user_id,))
        sender_username = self.cursor.fetchone()
        sender_username = sender_username[0] if sender_username else "Unknown"  # Extract username or set default

        # Fetch the selected peer's username from the database
        self.cursor.execute("SELECT p_username FROM patient_users WHERE p_id = ?", (receiver_id,))
        receiver_username = self.cursor.fetchone()
        receiver_username = receiver_username[0] if receiver_username else "Unknown"  # Extract username or set default

        # Retrieve previous messages exchanged between the logged-in user and the selected peer
        self.cursor.execute(
            """
            SELECT sender_id, message FROM peer_chat 
            WHERE (sender_id = ? AND receiver_id = ?) 
            OR (sender_id = ? AND receiver_id = ?)
            ORDER BY chat_id ASC
            """, 
            (self.logged_in_user_id, receiver_id, receiver_id, self.logged_in_user_id)
        )

        # Fetch all matching messages from the query
        messages = self.cursor.fetchall()

        # Clear the current message feed to refresh it with new messages
        self.message_feed.delete(0, 'end')

        # Loop through each message and display it in the chat feed
        for sender, message in messages:
            # Determine whether the sender is the logged-in user or the selected friend
            display_name = sender_username if sender == self.logged_in_user_id else receiver_username

            # Insert the message into the message feed with the sender's actual username
            self.message_feed.insert("end", f"{display_name}: {message}")

    def extract_user_id(self, user_info):
        """
        Extracts the user ID from the selected user info (format: 'Firstname Lastname (ID)').
        
        Parameters:
        - user_info (str): The selected user's string containing their name and ID.

        Returns:
        - int: The extracted user ID.
        - None: If extraction fails due to an incorrect format.
        
        """
        
        try:
            # get patient ID and remove any trailing spaces
            user_id = int(user_info.split("(")[-1][:-1])
            print(f"Extracted User ID: {user_id} from '{user_info}'")  # Debugging print in terminal
            return user_id
        except (IndexError, ValueError) as e:
            # handles any cases where the input is incorrect
            print(f"Error extracting user ID from '{user_info}': {e}")  # Debugging print in terminal
            return None

    def get_patient_users(self):
        """
        Retrieves all users from the 'patient_users' table and formats them as 'Firstname Lastname (ID)'.
        Returns:
            A list of str: A list of patient users in drop down
        """
        
        # SQL to fetch patients name and IDs
        self.cursor.execute("SELECT p_id, p_first_name, p_last_name FROM patient_users")
        # format the data into a row for dropdown list
        return [f"{row[1]} {row[2]} ({row[0]})" for row in self.cursor.fetchall()]

    def center_window(self, width, height):
        """
        Centers the window on the screen.
        """
        screen_width = self.peer.winfo_screenwidth()
        screen_height = self.peer.winfo_screenheight()
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.peer.geometry(f"{width}x{height}+{x}+{y}")

    def close(self):
        """
        Closes the database connection.
        """
        
        self.conn.close()


In [20]:
class UserProfile: # Displays well for patient to message eachother and to send counselor 
    def __init__(self, firstname, lastname, logged_in_user_id): # added argument for user profiles firstname and lastname to appear on welcome label
        """
        Window to appear when the user has created an account successfully or logged in successfully
        
        """
        # Initialize top level window
        self.profile = Toplevel()        
        self.profile.title(" Resource Treatment Center || Account")
        self.profile.geometry('800x800')
        self.center_window(800, 800)      # JS centers window
        self.profile.resizable(True, True)
        self.profile.config(background="light grey")
        
        # Create a Canvas widget to hold background image and label/button widgets
        self.canvas = Canvas(self.profile, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="fist-bump.png") # adjust image to display neutral colors
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        
        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)

	    # define Colors
        self.label_font_color = "navy"

	    # Label background color
        self.label_bg = "white"

	    # Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg=("navy")
        self.button_fg=("light gray")

	    # Diss Board Positng background color
        self.post_background = "light grey"
        
        # Title for the Profile window
        self.welcome_label = Label(self.profile, text=f"Welcome, {firstname} {lastname} to RTC Support Network!", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg) # SH added variables to display name on profile
        # place label on canvas
        self.canvas.create_window(400, 50, window=self.welcome_label)
        # create detailed message label
        self.detailMessage = Label(self.profile, text="RTC's Network Application where communication is always welcome! \n Enjoy chatting in the open discussion board or private message a friend or counselor at any moment!", fg=self.label_font_color, font=self.entry_font, bg=self.label_bg)
        # place label on canvas
        self.canvas.create_window(400, 150, window=self.detailMessage)
        
        
        # storing logged in user id as part of this instance for chat interfaces
        self.logged_in_user_id = logged_in_user_id
        # Peer Chat Page button
        self.peerChat_button = Button(self.profile, text="Private Message Peers", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.open_peer_chat)
        self.canvas.create_window(400, 250, window=self.peerChat_button)
        
        # Counselor Chat Page button
        self.counselorChat_button = Button(self.profile, text="Private Message a Counselor", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.open_counselor_chat)
        self.canvas.create_window(400, 350, window=self.counselorChat_button)
        """
        # Counselor Chat a patient Page button
        self.counselorChat_button = Button(self.profile, text="Counselor Message a Patient", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.open_counselor_chat)
        self.canvas.create_window(400, 350, window=self.counselorChat_button)
        """
        
        # Diss Board Page button
        # self.dissBoard_button = Button(self.profile, text="Open Discussion Board", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.open_diss_boardd)
        # self.canvas.create_window(400, 450, window=self.dissBoard_button)
        
    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.profile.winfo_screenwidth()
        screen_height = self.profile.winfo_screenheight()
                # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.profile.geometry(f"{width}x{height}+{x}+{y}") 
        
        
    # placeholder methods for the commands to open more windows
    def open_peer_chat(self):
        """
            Open Peer Chat Window when button is clicked via profile
        """
        PeerChat(db_path="support_network.db", logged_in_user_id=self.logged_in_user_id)
        
        
        
    def open_counselor_chat(self):
        """
            Open the Counselor Chat window when button is clicked via profile 
        """
        MessageCounselor(db_path="support_network.db", logged_in_user_id=self.logged_in_user_id)
        
        
    def open_diss_board(self):
        """
            Open the Discussion Board window when button is clicked via profile 
        """
        # DissBoard()
        pass


In [21]:
class LoginWindow(): # works well to validate username/password for patients
    def __init__(self):
        """
            Login Window  
            Created by Sammi Harper
            This is the login window of the application to allow users to log in and access the account features. 

        """
        # Initialize top level window
        self.login = Toplevel()
        self.login.title("Resource Treatment Center || LOGIN")
        self.login.geometry('500x500')
        self.center_window(500, 500)
        self.login.resizable(True, True)
        
        # Create a Canvas widget to hold background image and label/button widgets     
        self.canvas = Canvas(self.login, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="LOGIN_FIST_BUMP.png") # adjust image size to fit
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
    

        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)

        # Define Colors
        self.label_font_color = "navy"
        self.label_bg = "white"
        self.entry_bg = "light grey"

        # Button specs
        self.button_bg = "navy"
        self.button_fg = "light gray"

        # Title for the login window
        self.title_label = Label(self.login, text="Login to your Account", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(250, 75, window=self.title_label)

        # Username label and entry
        self.username_label = Label(self.login, text="Username:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 125, window=self.username_label)
        self.username_entry = Entry(self.login, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 125, window=self.username_entry)

        # Password label and entry
        self.password_label = Label(self.login, text="Password:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 175, window=self.password_label)
        self.password_entry = Entry(self.login, font=self.entry_font, bg=self.entry_bg, show="*")
        self.canvas.create_window(250, 175, window=self.password_entry)

        # Login button
        self.login_button = Button(self.login, text="Login", font=self.entry_font, bg=self.button_bg, fg=self.button_fg, command=self.validate_login)
        self.canvas.create_window(250, 225, window=self.login_button)   

    def center_window(self, width, height): # JS centers login window in center of screen
        screen_width = self.login.winfo_screenwidth()
        screen_height = self.login.winfo_screenheight()
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.login.geometry(f"{width}x{height}+{x}+{y}")
        
    def validate_login(self):
        """
        Validate the login credentials and open the User Profile if valid. Otherwise, display an error message.  
        """
        
        # Get username and password from the entries
        username = self.username_entry.get()
        password = self.password_entry.get()

        # Check if username and password exist
        user_info = self.check_credentials(username, password)
        
        if user_info:
            # Extract the user type and result
            user_type, result = user_info
            # Unpack the result tuple into ID, first name, and last name
            user_id, firstname, lastname = result

            if user_type == 'patient':  # If the user is a patient
                messagebox.showinfo("Login Success", f"Welcome {firstname} {lastname}!")
                self.login.destroy()
                UserProfile(firstname, lastname, logged_in_user_id=user_id)  # Open patient account window
            elif user_type == 'counselor':  # If the user is a counselor
                messagebox.showinfo("Login Success", f"Welcome {firstname} {lastname}!")
                self.login.destroy()
                UserProfile(firstname, lastname, logged_in_user_id=user_id)  # Open counselor account window
            else:
                # Handle unexpected user_type format
                messagebox.showerror("Login Failed", "Invalid user type detected")
        else:
            # If user_info is None, display error
            messagebox.showerror("Login Failed", "Invalid username or password")


    def check_credentials(self, username, password):
        """
        Validate the login values against the database. Returns the first name, last name, and ID if valid,
        otherwise returns None.
        """
        try:
            # Connect to the database
            conn = sqlite3.connect('support_network.db')  # Updated to match your CreateAccount class database
            cursor = conn.cursor()

            # Check if the username and password exist for patient users
            cursor.execute("SELECT p_id, p_first_name, p_last_name FROM patient_users WHERE p_username=? AND p_password=?", (username, password))
            result = cursor.fetchone()

            if result:
                conn.close()
                return ("patient", result)  # Return a tuple with the type 'patient' and the result (ID, first name, last name)

            # Check if the username and password exist for counselor users
            cursor.execute("SELECT c_id, c_first_name, c_last_name FROM counselor_users WHERE c_username=? AND c_password=?", (username, password))
            result = cursor.fetchone()

            if result:
                conn.close()
                return ("counselor", result)  # Return a tuple with the type 'counselor' and the result (ID, first name, last name)

            conn.close()
            return None  # If no valid result was found

        except sqlite3.Error as e:
            messagebox.showerror("Database Error", f"An error occurred: {e}")
            return None


In [22]:
class CreateAccount(): # validates details for patient account creation
    def __init__(self):
        """
        Create Account Window  
        Created by Sammi Harper
        This is the window fpr users to create an account to access the account features. 

        """
        # Initialize top level window
        self.CreateAccount = Toplevel()
        self.CreateAccount.title(" Resource Treatment Center || CREATE Account")
        self.CreateAccount.geometry('500x500')
        self.center_window(500, 500)      # JS centers window
        self.CreateAccount.resizable(True, True)
        self.CreateAccount.config(background="light grey")
        
        self.canvas = Canvas(self.CreateAccount, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="login_fist_bump.png") # adjust image size to fit
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        
        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)
        
        # define Colors
        self.label_font_color = "navy"
        
        # Label background color
        self.label_bg = "white" # SH changed label background color to white 
        self.entry_bg = "light grey"  # sh added entry background color

        # Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg=("navy")
        self.button_fg=("light gray")
        
        
        # title for the CreateAccount window
        self.title_label = Label(self.CreateAccount, text="Create your Account", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(250, 75, window=self.title_label)
        
        # Username label and entry
        self.username_label = Label(self.CreateAccount, text="Username:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 125, window=self.username_label)
        self.username_entry = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 125, window=self.username_entry) 
        
        # Password label and entry
        self.password_label = Label(self.CreateAccount, text="Password:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 160, window=self.password_label)
        self.password_entry = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg, show="*")
        self.canvas.create_window(250, 160, window=self.password_entry)
        
        # First name label and entry
        self.first_name = Label(self.CreateAccount, text="First Name:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 195, window=self.first_name)
        self.first_name = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 195, window=self.first_name)
        
        # Last name label and entry
        self.last_name = Label(self.CreateAccount, text="Last Name:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 230, window=self.last_name)
        self.last_name = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 230, window=self.last_name)
        
        # Email label and entry
        self.email = Label(self.CreateAccount, text="Email Address:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 265, window=self.email)
        self.email = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 265, window=self.email)
        
        # Phone Number label and entry
        self.phone_number = Label(self.CreateAccount, text="Phone Number:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 300, window=self.phone_number)
        self.phone_number = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 300, window=self.phone_number)
        
        #Counselor Name label and entry
        self.Counselor = Label(self.CreateAccount, text="Counselor's ID:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 335, window=self.Counselor)
        self.Counselor = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 335, window=self.Counselor)
        
        # Create Account button
        self.CreateAccount = Button(self.CreateAccount, text="Create Account", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.create_acc) # TODO: Add account check functionality and create the counselor screen
        self.canvas.create_window(250, 380, window=self.CreateAccount)
        
        
        # initialize database
        self.init_support_net_db()
        

    def init_support_net_db(self):
        """
        Initializes the SQLite database and creates necessary tables for patients and counselors.
        Ensures that the database connection is properly closed to prevent locking issues.
        """
        
        # SH had to add try and accept for code or DB locked and application broke
        try: 
            conn = sqlite3.connect('support_network.db')
            cursor = conn.cursor()
            patient_user = """CREATE TABLE IF NOT EXISTS patient_users (
                p_id INTEGER PRIMARY KEY NOT NULL,
                p_username TEXT UNIQUE NOT NULL,
                p_password TEXT NOT NULL,
                p_first_name TEXT NOT NULL,
                p_last_name TEXT NOT NULL,
                p_email TEXT UNIQUE NOT NULL,
                p_phone_number TEXT UNIQUE NOT NULL,
                c_id INTEGER NOT NULL,
                FOREIGN KEY(c_id) REFERENCES counselor_users(c_id))"""
                
                
            cursor.execute(patient_user) # SH changed the counselor last name to counselor id to be a foreign key referenceing the counselor table
            conn.commit()
        
                    
            # TODO: James --> add table to for counselors #JS 2/22 unsure if this is correct --> SH these are column names for the DB - they should reference the counselor details not the patient table. 
            # changed variables to have a c in front rather than a p -- SH
            counselor_user = """CREATE TABLE IF NOT EXISTS counselor_users (
                c_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                c_username TEXT UNIQUE NOT NULL,           
                c_password TEXT NOT NULL,
                c_first_name TEXT NOT NULL,
                c_last_name TEXT NOT NULL,
                c_email TEXT UNIQUE NOT NULL,
                c_phone_number TEXT UNIQUE NOT NULL)"""
            # TODO: James --> add cursor.execute command to add each counselor in DB (maybe like 5ish???)

            # Create the counselor_users table
            cursor.execute(counselor_user)
            
        except sqlite3.OperationalError as e:
            messagebox.showerror("Database Error", f"An error occurred: {e}")
        finally:
            conn.close()  # Ensure the connection is closed
        
    
    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.CreateAccount.winfo_screenwidth()
        screen_height = self.CreateAccount.winfo_screenheight()

        # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.CreateAccount.geometry(f"{width}x{height}+{x}+{y}")  
        
    def validate_input(self, username, password, first_name, last_name, email, phone_number, c_id):
        """
        This method validates the user input: username, password, first name, last name, email, phone number, and counselor's ID.
        Returns: True if all inputs are valid, False otherwise (False will display a message).
        Regex input validation site
        https://www.contentstack.com/docs/developers/create-content-types/validation-regex
        https://stackoverflow.com/questions/58774029/differences-between-re-match-re-search-re-fullmatch
        https://stackoverflow.com/questions/47877144/validate-name-using-python-regex 
        """
        
        if not all([username, password, first_name, last_name, email, phone_number, c_id]):
            messagebox.showerror("Error", "All fields are required!")
            return False
        
        # Username validation: must be at least 4 characters long (up to 12), have at least 1 letter/1 digit, and have no special characters
        username_regex = (r'^(?=.*[A-Za-z])(?=.*\d)[A-Za-z\d]{4,12}$')
        if not re.match(username_regex, username):
                messagebox.showerror("Error", "Username must be 4-12 characters long, have one digit and one letter, and have no special characters!")
                return False
        #Password validation: Must be at least 8 characters long and contain at least one letter and one digit
        password_regex = (r'^(?=.*[A-Za-z])(?=.*\d)[A-Za-z\d]{8,}$')
        #print("Password entered:", password)   # testing errors with password validation JS
        if not re.match(password_regex, password):
            messagebox.showerror("Error", "Password should be at least 8 characters long, and contain at least one letter and one number!")
            return False
        
        # Name validation: Only letters and spaces, between 2 and 30 characters
        name_regex = (r'^[A-Za-z\s]{2,30}$')
        if not re.match(name_regex, first_name) or not re.match(name_regex, last_name):
            messagebox.showerror("Error", "First and last names should only contain letters and spaces, and be between 2 and 30 characters.")
            return False
        
        # Email validation: Basic email format check
        email_regex = (r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
        if not re.match(email_regex, email):
            messagebox.showerror("Error", "Please enter a valid email address.")
            return False
        
        # Phone number validation: Only digits, and can be 10-15 characters long (optional + at the start)
        phone_regex = (r'^\+?[0-9]{10,15}$')
        if not re.match(phone_regex, phone_number):
            messagebox.showerror("Error", "Please enter a valid phone number (optional +, followed by 10-15 digits).")
            return False 
        
        # Counselor ID validation: Ensure it's numeric
        if not c_id.isdigit():  
            messagebox.showerror("Error", "Counselor's ID should be a number!")
            return False
        
        try:
            # connect to DB
            conn = sqlite3.connect("support_network.db")
            cursor = conn.cursor()

            # Check for existing username, email, and phone number
            cursor.execute("SELECT * FROM patient_users WHERE p_username = ? OR p_email = ? OR p_phone_number = ?", (username, email, phone_number))
            if cursor.fetchone():
                messagebox.showerror("Error", "Username, email, or phone number already exists!")
                conn.close()
                return False

            # Check if counselor ID exists in counselor_user table
            cursor.execute("SELECT * FROM counselor_users WHERE c_id = ?", (c_id,))
            if not cursor.fetchone():
                messagebox.showerror("Error", "Counselor's ID not found!")
                conn.close()
                return False

            conn.close()
            
        except sqlite3.Error as e:  # Handle any SQLite database errors
            messagebox.showerror("Database Error", f"An error occurred: {e}")
            return False
        
        return True



    def create_acc(self):
        """
        Collects user input, validates the data, and inserts a new patient record into the database.
        Ensures proper error handling to prevent database integrity issues.
        """
        # .get command must be listed as the Entry Box name from creating the input box 
        username = self.username_entry.get()
        password = self.password_entry.get()
        first_name = self.first_name.get()
        last_name = self.last_name.get()
        email = self.email.get()
        phone_number = self.phone_number.get()
        c_id = self.Counselor.get()

        # Validate the input fields and create the account if valid
        if not self.validate_input(username, password, first_name, last_name, email, phone_number, c_id):
            return # Stop execution if validation fails
        
        try:
            conn = sqlite3.connect("support_network.db")
            cursor = conn.cursor()
            # Insert the new patient record into the patient_users table
            cursor.execute('''INSERT INTO patient_users (p_username, p_first_name, p_last_name, p_email, p_phone_number, p_password, c_id)
                            VALUES (?, ?, ?, ?, ?, ?, ?)''', (username, first_name, last_name, email, phone_number, password, c_id))
            conn.commit()
            conn.close()
            messagebox.showinfo("Success", "Account created successfully!")
            self.CreateAccount.destroy()
        # Push error message if validation fails or other DB error occurs
        except (sqlite3.IntegrityError, sqlite3.Error) as e:
            messagebox.showerror("Error", f"An error occurred: {e}")



In [23]:
class LoginCounselorAccount:
    """
    This class manages the login functionality for counselors and provides a chat interface
    for communication with patients.
    """
    def __init__(self, db_path="support_network.db"):
        """
        Initializes the login window and sets up the counselor chat table in the database.
        """
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.counselor_chat_table()

        # Creating the login window
        self.CounselorPortal = Toplevel()
        self.CounselorPortal.title("Resource Treatment Center || Counselor Login")
        self.CounselorPortal.geometry('500x500')
        self.CounselorPortal.resizable(True, True)
        self.center_window(500, 500)

        # Create a Canvas widget to hold background image and label/button widgets     
        self.canvas = Canvas(self.CounselorPortal, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="login_fist_bump.PNG") # adjust image size to fit
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")

        # Create a Frame for login UI (positioned on top of background)
        self.login_frame = Frame(self.CounselorPortal, bg="white", bd=1, relief=None)
        self.login_frame.place(relx=0.5, rely=0.3, anchor="center")  # Adjusted position

        # UI Elements for login
        Label(self.login_frame, text="Counselor Login", font=("Arial", 14), bg="white").pack(pady=10)

        Label(self.login_frame, text="Username:", bg="white").pack()
        self.username_entry = Entry(self.login_frame)
        self.username_entry.pack(padx=20, pady=5)

        Label(self.login_frame, text="Password:", bg="white").pack()
        self.password_entry = Entry(self.login_frame, show="*")
        self.password_entry.pack(padx=20, pady=5)

        Button(self.login_frame, text="LOGIN", command=self.validate_login).pack(pady=10)
        
    def center_window(self, width, height): # JS centers login window in center of screen
        screen_width = self.CounselorPortal.winfo_screenwidth()
        screen_height = self.CounselorPortal.winfo_screenheight()
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.CounselorPortal.geometry(f"{width}x{height}+{x}+{y}")

    
    def validate_login(self):
        """
        Validates counselor login by checking credentials against the database.
        """
        username = self.username_entry.get().strip()
        password = self.password_entry.get().strip()
        
        if not username or not password:
            # Error message to display if both username and password are not provided
            messagebox.showerror("Login Failed", "Please enter both username and password.", parent=self.CounselorPortal)
            return

        try:
            # Connect to the database and validate the credentials
            conn = sqlite3.connect("support_network.db")
            cursor = conn.cursor()
            # Query the counselor_users table for matching credentials
            cursor.execute("SELECT c_id, c_first_name, c_last_name, c_username FROM counselor_users WHERE c_username = ? AND c_password = ?", (username, password))
            counselor = cursor.fetchone()
            conn.close()

            if counselor:
                # Set the counselor's ID, name, and username for future use
                self.counselor_id = counselor[0]
                self.logged_in_user_id = counselor[0]  
                self.counselor_name = f"{counselor[1]} {counselor[2]}"
                self.counselor_username = counselor[3]

                # Message Box to welcome counselor upon successful login
                messagebox.showinfo("Login Successful", f"Welcome, {self.counselor_name}!", parent=self.CounselorPortal)
                
                # destroy login box so credentials are not exposed after logging in
                self.login_frame.destroy()

                # Call chat window to open after login
                self.open_chat_window()

            else:
                # Error message to display if username and password are not found in the database
                messagebox.showerror("Login Failed", "Invalid Username or Password", parent=self.CounselorPortal)
                
        except sqlite3.Error as e:
            # Display error message if any database error occurs during login validation
            messagebox.showerror("Database Error", f"An error occurred: {e}", parent=self.CounselorPortal)

    
    def counselor_chat_table(self):
        """
        Creates the counselor_chat table if it does not already exist.
        """
        self.cursor.execute(''' 
            CREATE TABLE IF NOT EXISTS counselor_chat( 
                chat_id INTEGER PRIMARY KEY AUTOINCREMENT,
                sender_id INTEGER NOT NULL,
                receiver_id INTEGER NOT NULL,
                message TEXT NOT NULL,
                is_read INTEGER DEFAULT 0,
                FOREIGN KEY (sender_id) REFERENCES counselor_users(c_id),
                FOREIGN KEY (receiver_id) REFERENCES patient_users(p_id)
            )
        ''')
        self.conn.commit()

    def open_chat_window(self):
        """
        Opens a new chat window for counselors to communicate with patients.
        """
        # Creating toplevel window for counselor portal
        self.chat_window = Toplevel(self.CounselorPortal)
        self.chat_window.title("Counselor Portal")
        self.chat_window.geometry("500x500")

        # Center the chat window on screen
        self.chat_window.update_idletasks()
        window_width = self.chat_window.winfo_width()
        window_height = self.chat_window.winfo_height()
        screen_width = self.chat_window.winfo_screenwidth()
        screen_height = self.chat_window.winfo_screenheight()
        x = (screen_width // 2) - (window_width // 2)
        y = (screen_height // 2) - (window_height // 2)
        self.chat_window.geometry(f"{window_width}x{window_height}+{x}+{y}")

        # Create a Canvas widget
        self.canvas = Canvas(self.chat_window, width=500, height=500)
        self.canvas.pack(fill="both", expand=True)

        # Create a Frame inside the Canvas to hold UI elements
        self.chat_frame = Frame(self.canvas)
        self.canvas_window = self.canvas.create_window(250, 250, window=self.chat_frame, anchor="center")
        
        

        # Welcome Label and Patient Dropdown
        Label(self.chat_frame, text=f"Welcome, {self.counselor_name}", font=("Arial", 12)).pack(pady=5, anchor="center")
        Label(self.chat_frame, text="Select a Patient:").pack(anchor="center")

        # Create String Variable to store selected patient's name
        self.selected_patient = StringVar()

        # Dropdown menu for patient selection
        self.patient_dropdown = ttk.Combobox(self.chat_frame, textvariable=self.selected_patient)
        self.patient_dropdown.pack(pady=5)

        # Load patients into the dropdown
        patient_list = self.get_patient_users()
            
        if patient_list:
            self.patient_dropdown["values"] = patient_list
        else:
            self.patient_dropdown["values"] = ["No Patients Found"]  # Handle empty list case

        # Button to load prior messages
        Button(self.chat_frame, text="Load Messages", command=self.load_previous_messages).pack(pady=5, anchor="center")

        # Message listbox (with scrollbar)
        self.message_listbox = Listbox(self.chat_frame, width=50, height=15)
        self.message_listbox.pack(pady=5, anchor="center")

        # Message entry field
        Label(self.chat_frame, text="Message:").pack(anchor="center")
        self.chat_entry = Entry(self.chat_frame, width=50)
        self.chat_entry.pack(pady=5)

        # Send button
        Button(self.chat_frame, text="Send", command=lambda: self.send_patient_message("support_network.db", self.counselor_id)).pack(pady=10, anchor="center")

    
    def load_previous_messages(self):
        """
        Loads previous messages between the logged-in user and the selected counselor.
        Retrieves messages from the database and displays them in the message feed 
        using usernames.
        """
        
        # Get the selected peer's information (usually in a format like "Username (ID)")
        receiver_info = self.selected_patient.get()

        # Extract the user ID of the selected peer from the information
        receiver_id = self.extract_patient_id(receiver_info)

        # If no receiver is selected or the user is not logged in, do nothing
        if not receiver_id or self.logged_in_user_id is None:
            return

        # Fetch the logged-in user's username from the database
        self.cursor.execute("SELECT c_username FROM counselor_users WHERE c_id = ?", (self.logged_in_user_id,))
        sender_username = self.cursor.fetchone()
        sender_username = sender_username[0] if sender_username else "Unknown"  # Extract username or set default

        # Fetch the selected counselor's username from the database
        self.cursor.execute("SELECT p_username FROM patient_users WHERE p_id = ?", (receiver_id,))
        receiver_username = self.cursor.fetchone()
        receiver_username = receiver_username[0] if receiver_username else "Unknown"  # Extract username or set default

        # Retrieve previous messages exchanged between the logged-in user and the selected peer
        self.cursor.execute(
            """
            SELECT sender_id, message FROM counselor_chat 
            WHERE (sender_id = ? AND receiver_id = ?) 
            OR (sender_id = ? AND receiver_id = ?)
            ORDER BY chat_id ASC
            """, 
            (self.logged_in_user_id, receiver_id, receiver_id, self.logged_in_user_id)
        )

        # Fetch all matching messages from the query
        messages = self.cursor.fetchall()

        # Clear the current message feed to refresh it with new messages
        self.chat_entry.delete(0, 'end')

        # Loop through each message and display it in the chat feed
        for sender, message in messages:
            # Determine whether the sender is the logged-in user or the selected friend
            display_name = sender_username if sender == self.logged_in_user_id else receiver_username

            # Insert the message into the message feed with the sender's actual username
            self.message_listbox.insert("end", f"{display_name}: {message}")
    
    def get_patient_users(self):
        """
        Retrieves all users from the 'patient_users' table and formats them as 'Firstname Lastname (ID)'.
        Returns:
            A list of str: A list of patient users in drop down
        """
        
        # SQL to fetch counselor name and IDs
        self.cursor.execute("SELECT p_id, p_first_name, p_last_name FROM patient_users")
        # format the data into a row for dropdown list
        return [f"{row[1]} {row[2]} ({row[0]})" for row in self.cursor.fetchall()]
    
    def send_patient_message(self, db_path, logged_in_user_id):
        """
        Sends the message to the selected patient and updates the message feed.
        """
        self.db_path = db_path
        self.logged_in_user_id = logged_in_user_id

        # Ensure a patient is selected before proceeding
        receiver_info = self.selected_patient.get().strip()
        if not receiver_info:
            messagebox.showerror("Error", "Please select a patient before sending a message.")
            return

        # Extract the patient's ID
        receiver_id = self.extract_patient_id(receiver_info)
        if receiver_id is None:
            messagebox.showerror("Error", "Invalid patient selection. Please select a valid patient.")
            return

        # Get message input
        message = self.chat_entry.get().strip()
        if not message:
            messagebox.showerror("Error", "Please enter a message before sending.")
            return

        # Fetch sender's username
        self.cursor.execute("SELECT c_username FROM counselor_users WHERE c_id = ?", (self.logged_in_user_id,))
        sender_username = self.cursor.fetchone()
        sender_username = sender_username[0] if sender_username else "Unknown"

        # Fetch receiver's username
        self.cursor.execute("SELECT p_username FROM patient_users WHERE p_id = ?", (receiver_id,))
        receiver_username = self.cursor.fetchone()
        receiver_username = receiver_username[0] if receiver_username else "Unknown"

        # Insert message into the database
        self.cursor.execute(
            "INSERT INTO counselor_chat (sender_id, receiver_id, message, is_read) VALUES (?, ?, ?, 0)",
            (self.logged_in_user_id, receiver_id, message)
        )
        self.conn.commit()

        # Update message feed
        self.message_listbox.insert("end", f"{sender_username}: {message}")
        self.chat_entry.delete(0, 'end')
        
    def extract_patient_id(self, user_info):
        """
        Extracts the patient ID from the selected patient info (format: 'Firstname Lastname (ID)').

        Returns:
            int: Extracted patient ID.
            None: If extraction fails.
        """
        if not user_info:
            print("Error: No patient selected.")
            return None  # Avoid processing empty strings
        
        try:
            # Extracts the numeric ID from the format "Firstname Lastname (ID)"
            user_id = int(user_info.split("(")[-1][:-1])
            print(f"Extracted User ID: {user_id} from '{user_info}'")
            return user_id
        except (IndexError, ValueError) as e:
            print(f"Error extracting user ID from '{user_info}': {e}")
            return None

    def get_patient_users(self):
        """
        Retrieves all users from the 'patient_users' table and formats them as 'Firstname Lastname (ID)'.
        Returns:
            A list of str: A list of patient users in drop down
        """
        
        # SQL to fetch counselor name and IDs
        self.cursor.execute("SELECT p_id, p_first_name, p_last_name FROM patient_users")
        # format the data into a row for dropdown list
        return [f"{row[1]} {row[2]} ({row[0]})" for row in self.cursor.fetchall()]

    def center_window(self, width, height):
        """
        Centers the window on the screen.
        """
        screen_width = self.CounselorPortal.winfo_screenwidth()
        screen_height = self.CounselorPortal.winfo_screenheight()
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.CounselorPortal.geometry(f"{width}x{height}+{x}+{y}")

    def close(self):
        """
        Closes the database connection.
        """
        
        self.conn.close()

In [ ]:
"""
Main Window GUI 
Created by Sammi Harper
This is the main window of the application to allow users to log in or create an account. 

"""

class MainWindow():
    def __init__(self, master_window):
        # Setting and creating window to be resizable by user
        self.master = master_window
        self.master.title(" Resource Treatment Center ")
        self.master.geometry('800x800')
        self.center_window(800, 800)      # JS centers window
        self.master.resizable(True, True)

        self.canvas = Canvas(self.master, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="fist-bump.png") # adjust image size to fit
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        # Application font
        self.label_font = ("Helvetica", 25)

        # Sub-label font sizes
        self.sub_label_font = "Helvetica", 16

        # Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg = "navy"
        self.button_fg = "light gray"

        # Top label font size and color
        self.top_font = Font(family="Helvetica", size=35, weight="bold")
        # Color for top label font
        self.color_font = "Navy"
        self.bg = "white"


        # Create a welcome label with title and subtitle on the top of the window
        title = Label(self.master, text="Support Network Application", fg=self.color_font, bg=self.bg,
                    font=self.top_font)
        # Position the label on the top of the window using place()
        title.place(relx=0.5, rely=0.1, anchor="center")

        subTtitle = Label(self.master, text="Please log in or create a new account", fg=self.color_font, font=self.sub_label_font, bg=self.bg)
        # Position the sub-title below the title using place()
        subTtitle.place(relx=0.5, rely=0.2, anchor="center")

        # CREATE LOGIN BUTTON
        login_button = tk.Button(self.master, text="Login to Account", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=LoginWindow)
        # Position the login button using place()
        login_button.place(relx=0.35, rely=0.5, anchor="center")

        # CREATE CREATE ACCOUNT BUTTON
        create_account_button = tk.Button(self.master, text="Create an Account", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=CreateAccount)
        # Position the create account button using place()
        create_account_button.place(relx=0.65, rely=0.5, anchor="center")
        
        # counselor account button
        counselor_account_button = tk.Button(self.master, text="Counselor Account", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=LoginCounselorAccount)
        # Position the counselor account button using place()
        counselor_account_button.place(relx=0.5, rely=0.7, anchor="center")

    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.master.winfo_screenwidth()
        screen_height = self.master.winfo_screenheight()

        # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.master.geometry(f"{width}x{height}+{x}+{y}")


def main():
    root = tk.Tk()
    window = MainWindow(root)
    root.mainloop()

if __name__ == '__main__':
    main()


Extracted User ID: 2 from 'Robby Robson (2)'
Extracted User ID: 2 from 'Robby Robson (2)'
Extracted User ID: 5 from 'Elli May (5)'
Extracted User ID: 5 from 'Elli May (5)'
Extracted User ID: 2 from 'Robby Robson (2)'
